In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale

# Download the date
https://archive.ics.uci.edu/ml/machine-learning-databases/00577/codon_usage.csv.zip


In [ ]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00577/codon_usage.csv.zip

In [ ]:
# !unzip codon_usage.csv.zip

# Data Preprocessing

In [ ]:
def read_data(file_name):
    """
    Reads in a csv file and returns a dataframe
    """
    return pd.read_csv(file_name, low_memory=False)

dataset = read_data('codon_usage.csv')
df = dataset.copy()

In [ ]:
# dataset = read_data('codon_usage.csv')
dataset.describe()

In [ ]:
dataset

In [ ]:
# dataset.dropna(inplace=True)

dataset['Kingdom'].value_counts()

In [ ]:
print(dataset.isnull().sum())

In [ ]:
dataset['Kingdom'].value_counts().plot(kind='bar')

In [ ]:
dataset['Kingdom'].unique()

In [ ]:
dataset['Kingdom'] = dataset['Kingdom'].map({   'arc': 0, 'bct': 1, 
                                                'phg': 2, 'plm': 3, 
                                                'pln': 4, 'inv': 5, 
                                                'vrt': 6, 'mam': 7,
                                                'rod': 8, 'pri': 9, 'vrl':10})

In [ ]:
dataset['Kingdom'].value_counts().plot(kind='bar')

In [ ]:
dataset = dataset.drop(dataset[dataset['SpeciesID']==353569].index)
dataset = dataset.drop(dataset[dataset['SpeciesID']==1238].index)


In [ ]:
X = dataset.iloc[:, 5:].values.astype(float)
y = dataset.iloc[:, 0]

In [ ]:
y

In [ ]:
X

In [ ]:
dataset['DNAtype'].value_counts().plot(kind='bar')

In [ ]:
class GaussianNaiveBayes:
  def __init__(self, X, y, laplace_smoothing_value, discrete, laplace, classes):
    # update your Naive Bayes class functions to account for a discrete classification using what we have learned in the class. 
    self.X =X # training data
    self.y = y # y is the original classes
    self.laplace_smoothing_value = laplace_smoothing_value # laplace smoothing value
    self.discrete = discrete # boolean variable to indicate if the data is discrete or continuous
    self.laplace = laplace  # boolean for laplace smoothing
    self.classes = classes  # this is the number of classes in the data



  def splitData(self):
    X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.3, random_state=1)
    print('Training data size: ', X_train.shape) # (890, 10)
    print('Test data size: ', X_test.shape) # (190, 10)
    return X_train, X_test, y_train, y_test # return the training and test data 



  def fitDistribution(self, data):

    mean = np.mean(data) # mean of the data
    std = np.std(data) # standard deviation of the data

    dist = norm(mean, std) # create a normal distribution with the mean and standard deviation 

    return dist # return the distribution



  def laplaceSmoothing(self, classes, length):
    # print('type 0 ', type(class0))
    # print('size 0 ', class0.size)
    # print('type 1 ', type(class1))
    # print('size 1 ', class1.size)
    # smooth0 = (class0 + self.laplace_smoothing_value) / ( (class0.size) + self.laplace_smoothing_value * self.n_features)
    # smooth1 = (class1 + self.laplace_smoothing_value) / ( (class1.size) + self.laplace_smoothing_value * self.n_features)
    # smooth2 = (class2 + self.laplace_smoothing_value) / ( (class2.size) + self.laplace_smoothing_value * self.n_features)
    # smooth3 = (class3 + self.laplace_smoothing_value) / ( (class3.size) + self.laplace_smoothing_value * self.n_features)
    # smooth4 = (class4 + self.laplace_smoothing_value) / ( (class4.size) + self.laplace_smoothing_value * self.n_features)
    # smooth5 = (class5 + self.laplace_smoothing_value) / ( (class5.size) + self.laplace_smoothing_value * self.n_features)
    # smooth6 = (class6 + self.laplace_smoothing_value) / ( (class6.size) + self.laplace_smoothing_value * self.n_features)
    # smooth7 = (class7 + self.laplace_smoothing_value) / ( (class7.size) + self.laplace_smoothing_value * self.n_features)
    # smooth8 = (class8 + self.laplace_smoothing_value) / ( (class8.size) + self.laplace_smoothing_value * self.n_features)
    # smooth9 = (class9 + self.laplace_smoothing_value) / ( (class9.size) + self.laplace_smoothing_value * self.n_features)
    # smooth10 = (class10 + self.laplace_smoothing_value) / ( (class10.size) + self.laplace_smoothing_value * self.n_features)
    smooths = [] # create an empty list to store the smoothed values
    for i in range(length): 
      # add the laplace smoothing value to the class and divide by the size of the class plus the laplace smoothing value times the number of features
      smooths[i] = (classes[i] + self.laplace_smoothing_value) / ( (classes[i].size) + self.laplace_smoothing_value * self.n_features) 
    
    # return the smoothed values
    
    return smooths 
  



  def evaluate(self, y, y_predicted):
    '''

      Takes original classes and predicted classes as input

      Return the values of precision, recall and accuracy
    
    '''
    y = ( y==1 )
    y_predicted = ( y_predicted == 1 )

    precision = (y&y_predicted).sum() / y_predicted.sum()
    recall = (y&y_predicted).sum() / y.sum()
    accuracy = (y==y_predicted).sum() / y.size


    return precision, recall, accuracy



  # def probability(self, X, dist1, dist2, prior):
  #   return prior * ( dist1.pdf(X[0]) *  dist2.pdf(X[1]))


  def probability(self, data):
    # py0 = 1
    # py1 = 1
    py = np.ones(11) # create an array of ones to store the probabilities


    if self.discrete:
      for i in range(self.n_features): # for each feature
        # py[0] *= self.laplaceSmoothing(self.class0[i], self.n_features)

        for j in range(11): # for each class
          print('i, j ', i, j) # print the feature and class
          # multiply the probability by the probability of the feature given the class
          py[j] *= self.features['X'+str(i)+str(j)] 

        # py0 *= self.features['X'+str(i)+str(data[i])+'0']
        # py1 *= self.features['X'+str(i)+str(data[i])+'1']
        # py2 *= self.features['X'+str(i)+str(data[i])+'2']
        # py3 *= self.features['X'+str(i)+str(data[i])+'3']
        # py4 *= self.features['X'+str(i)+str(data[i])+'4']
        # py5 *= self.features['X'+str(i)+str(data[i])+'5']

    else:
      for i in range(self.n_features):
        for j in range(11):
          py[j] *= self.features['X'+str(i)+str(j)].pdf(data[i])
        # py0 *= self.features['X'+str(i)+'0'].pdf(data[i])
        # py1 *= self.features['X'+str(i)+'1'].pdf(data[i])

    for i in range(11):
      py[i] *= self.prior[i]
    
    return  py



  def laplaceProbability(self, data):
    # py0 = 1
    # py1 = 1

    py = np.ones(11) # create an array of ones to store the probabilities

    if self.discrete:
      for i in range(self.n_features):
        print(data[i])
        print('X'+str(i)+'0')
        smooths = self.laplaceSmoothing(self.features, self.n_features)
        class0 = self.features['X'+str(i)+'0']
        smooth0 , smooth1, smooth2, smooth3, smooth4, smooth5, smooth6, smooth7, smooth8, smooth9, smooth10 = self.laplaceSmoothing( class0 = self.features['X'+str(i)+'0'], class1 = self.features['X'+str(i)+'1'], class2 = self.features['X'+str(i)+'2'], 
                                                    class3 = self.features['X'+str(i)+'3'], class4 = self.features['X'+str(i)+'4'], class5 = self.features['X'+str(i)+'5'], 
                                                    class6 = self.features['X'+str(i)+'6'], class7 = self.features['X'+str(i)+'7'], class8 = self.features['X'+str(i)+'8'], 
                                                    class9 = self.features['X'+str(i)+'9'], class10 = self.features['X'+str(i)+'10'])
        print(smooth0, smooth1, smooth2, smooth3, smooth4, smooth5, smooth6, smooth7, smooth8, smooth9, smooth10)
        py0 *= smooth0
        py1 *= smooth1
        py2 *= smooth2
        py3 *= smooth3
        py4 *= smooth4
        py5 *= smooth5
        py6 *= smooth6
        py7 *= smooth7
        py8 *= smooth8
        py9 *= smooth9
        py10 *= smooth10
    
    else:
      for i in range(self.n_features):
        pdf0 = self.features['X'+str(i)+'0'].pdf(data[i])
        pdf1 = self.features['X'+str(i)+'1'].pdf(data[i])
        pdf2 = self.features['X'+str(i)+'2'].pdf(data[i])
        pdf3 = self.features['X'+str(i)+'3'].pdf(data[i])
        pdf4 = self.features['X'+str(i)+'4'].pdf(data[i])
        pdf5 = self.features['X'+str(i)+'5'].pdf(data[i])
        pdf6 = self.features['X'+str(i)+'6'].pdf(data[i])
        pdf7 = self.features['X'+str(i)+'7'].pdf(data[i])
        pdf8 = self.features['X'+str(i)+'8'].pdf(data[i])
        pdf9 = self.features['X'+str(i)+'9'].pdf(data[i])
        pdf10 = self.features['X'+str(i)+'10'].pdf(data[i])
        smooth0 , smooth1, smooth2, smooth3, smooth4, smooth5, smooth6, smooth7, smooth8, smooth9, smooth10  = self.laplaceSmoothing(pdf0, pdf1, pdf2, pdf3, pdf4, pdf5, pdf6, pdf7, pdf8, pdf9, pdf10)
        py0 *= smooth0
        py1 *= smooth1
        py2 *= smooth2
        py3 *= smooth3
        py4 *= smooth4
        py5 *= smooth5
        py6 *= smooth6
        py7 *= smooth7
        py8 *= smooth8
        py9 *= smooth9
        py10 *= smooth10


    return  py0 * self.prior_0, py1 * self.prior_1, py2 * self.prior_2, py3 * self.prior_3, py4 * self.prior_4, py5 * self.prior_5, py6 * self.prior_6, py7 * self.prior_7, py8 * self.prior_8, py9 * self.prior_9, py10 * self.prior_10                                                                            




  def fit(self):

    self.X_train, self.X_test, self.y_train, self.y_test = self.splitData()
    X_training = self.X_train.copy()
    for i in range(self.classes):
      X_training[i] = self.X_train[self.y_train == i]
      self.prior[i] = len(X_training[i]) / len(self.X_train)

    # X0_train = self.X_train[self.y_train == 0]
    # X1_train = self.X_train[self.y_train == 1]
    # X2_train = self.X_train[self.y_train == 2]
    # X3_train = self.X_train[self.y_train == 3]
    # X4_train = self.X_train[self.y_train == 4]
    # X5_train = self.X_train[self.y_train == 5]
    # X6_train = self.X_train[self.y_train == 6]
    # X7_train = self.X_train[self.y_train == 7]
    # X8_train = self.X_train[self.y_train == 8]
    # X9_train = self.X_train[self.y_train == 9]
    # X10_train = self.X_train[self.y_train == 10]

    for i in range(self.classes):
      self.prior[i] = len(X_training[i]) / len(self.X_train)

    # self.prior_1 = len(X1_train) / len(self.X_train)
    # self.prior_0 = len(X0_train) / len(self.X_train)
    # self.prior_2 = len(X2_train) / len(self.X_train)
    # self.prior_3 = len(X3_train) / len(self.X_train)
    # self.prior_4 = len(X4_train) / len(self.X_train)
    # self.prior_5 = len(X5_train) / len(self.X_train)
    # self.prior_6 = len(X6_train) / len(self.X_train)
    # self.prior_7 = len(X7_train) / len(self.X_train)
    # self.prior_8 = len(X8_train) / len(self.X_train)
    # self.prior_9 = len(X9_train) / len(self.X_train)
    # self.prior_10 = len(X10_train) / len(self.X_train)
    


    self.n_features = self.X_train.shape[1]
    print('Number of features: ', self.n_features)
    self.features = {}
    
    if self.discrete:

      print('Discrete Data')

      for i in range(self.n_features):
        for j in range(self.n_classes):
          for k in range(self.n_classes):
            self.features['X'+str(i)+str(j)+str(k)] = ( X_training[i][X_training[:,i] == j] ) / len(X_training[i])

        # self.features['X'+str(i)+'00'] = ( (X0_train[ X0_train[:,i]==0 ]).sum() ) / len(X0_train)
        # self.features['X'+str(i)+'01'] = ( (X0_train[ X0_train[:,i]==0 ]).sum() ) / len(X1_train)
        # self.features['X'+str(i)+'10'] = ( (X0_train[ X0_train[:,i]==1 ]).sum() ) / len(X0_train)
        # self.features['X'+str(i)+'11'] = ( (X0_train[ X0_train[:,i]==1 ]).sum() ) / len(X1_train)

    
    else:

      print('Continuous Data')

      for i in range(self.n_features):
        for j in range(self.n_classes):
          self.features['X'+str(i)+str(j)] = self.fitDistribution(X_training[j][:,i])
          
        # self.features['X'+str(i)+'0'] = self.fitDistribution(X0_train[:, i])
        # self.features['X'+str(i)+'1'] = self.fitDistribution(X1_train[:, i])


    # self.X00_dist = self.fitDistribution(X0_train[:, 0])
    # self.X01_dist = self.fitDistribution(X0_train[:, 1])

    # self.X10_dist = self.fitDistribution(X1_train[:, 0])
    # self.X11_dist = self.fitDistribution(X1_train[:, 1])

  
  
  def predict(self):
    count = 0
    y_predicted = []
    y = []
    for sample, target in zip(self.X_test, self.y_test):
      # py0 = self.probability(sample, self.X00_dist, self.X01_dist, self.prior_0)
      # py1 = self.probability(sample, self.X10_dist, self.X11_dist, self.prior_1)

      # print("P(y=0|%s = %.3f" % (sample, py0*100))
      # print("P(y=1|%s = %.3f" % (sample, py1*100))

      if self.laplace:
        py0, py1 = self.laplaceProbability(sample)
      
      else:
        py0, py1 = self.probability(sample)

      print("Model predicted class {} and truth was {}".format(np.argmax([py0,py1]), target))
      y_predicted.append(np.argmax([py0,py1]))
      y.append(target)
      # np.append(y_predicted, np.argmax([py0,py1]))
      # np.append(y, target)
      # y = target
      if np.argmax([py0,py1]) != target :
        count+=1

      # precision, recall, accuracy = self.evaluate(y, y_predicted)
      # print("precision:", precision)
      # print("recall:", recall)
      # print("accuracy:", accuracy)

    
    print(count)
    # print(y)
    y_predicted = np.array(y_predicted)
    y = np.array(y)

    precision, recall, accuracy = self.evaluate(y, y_predicted)
    print("precision:", precision)
    print("recall:", recall)
    print("accuracy:", accuracy)
    print("f1 score:", 2*(precision*recall)/(precision+recall))
    print(set(y_predicted))

      

  



In [ ]:
clf = GaussianNaiveBayes(X,y, laplace_smoothing_value=0.5, discrete=False, laplace=True, classes=2)

In [ ]:
clf.fit()

In [ ]:
clf.predict()